In [1]:
import scipy.io as sio
import numpy as np
import scipy.stats as stats
import random
import matplotlib.pyplot as plt
import sklearn.feature_extraction.text as sktext

In [2]:
# Load data, the data used in this script is SPAM data set.
# The object 'data' is a dic that contains np.array 'training_data' and 'training_labels'.
matfn='spam_data.mat'
data=sio.loadmat(matfn)

In [4]:
# Split the training set and validation set
n_feature=data['training_data'].shape[1]
label=np.unique(data['training_labels'][0])
i=True
while i is True:
    valid_ind=random.sample(range(len(data['training_data'])),round(len(data['training_data'])*0.2))
    train_ind=[i for i in range(len(data['training_data'])) if i not in valid_ind]    
    valid_set=data['training_data'][valid_ind]
    train_set=data['training_data'][train_ind]
    valid_label=data['training_labels'][0][valid_ind]
    train_label=data['training_labels'][0][train_ind]        
    i= (False in [i in valid_label for i in label]) | (False in [i in train_label for i in label])

In [5]:
class Nodes:
    def __init__(self,split_rule=(-1,None),label=None,left=None,right=None):
        self.split_rule=split_rule # column index and criteria
        self.label=label # leaf node label
        self.left=left # nodes (<)
        self.right=right # nodes (>=)
        self.parent=None
        self.depth=1
        self.train_data=None
        self.train_labels=None
        
    def GrowTree(self, max_depth, train_data, train_labels,random_feature=-1):
        if (sum(train_labels)/len(train_labels))>0.95 or (sum(train_labels)/len(train_labels))<0.05 or len(train_labels)<10:
            self.label=round((sum(train_labels)/len(train_labels)))
            self.train_data=train_data
            self.train_labels=train_labels
        elif max_depth>0 and self.depth>max_depth:
            self.label=round((sum(train_labels)/len(train_labels)))
            self.train_data=train_data
            self.train_labels=train_labels             
        else:
            if random_feature==-1:
                self.split_rule=best_split(train_data,train_labels)
            else:
                sample_feature=random.sample(range(train_data.shape[1]),random_feature)
                sample_data=train_data[...,sample_feature]
                random_split_rule=best_split(sample_data,train_labels)
                self.split_rule=(sample_feature[random_split_rule[0]],random_split_rule[1])          
            #find best split rule, input train_data, train_labels, return a tuple
            if self.split_rule==(-1,None):
                self.label=round((sum(train_labels)/len(train_labels)))
                self.train_data=train_data
                self.train_labels=train_labels                 
            else:
                left_ind=train_data[...,self.split_rule[0]]<self.split_rule[1]
                right_ind=train_data[...,self.split_rule[0]]>=self.split_rule[1]
                self.left=Nodes()
                self.right=Nodes()
                self.left.parent=self
                self.left.depth=self.depth+1
                self.right.parent=self
                self.right.depth=self.depth+1
                self.left.GrowTree(max_depth,train_data[left_ind], train_labels[left_ind])
                self.right.GrowTree(max_depth,train_data[right_ind], train_labels[right_ind])

In [6]:
def best_split(train_data,train_labels):
    n_feature=len(train_data[0])
    H_after=[]
    ij_value=[]
    for i in range(n_feature):
        sort_ind=train_data[...,i].argsort()
        feature=train_data[...,i][sort_ind]
        label=train_labels[sort_ind]        
        for j in range(len(label)-1):
            if feature[j]!=feature[j+1]:
                n_left=j+1
                n_right=len(label)-n_left
                left_1=sum(label[0:n_left])
                right_1=sum(label[n_left:len(label)])
                p1_left=(left_1/n_left) or 0.0001
                p0_left=(1-p1_left) or 0.0001
                p1_right=(right_1/n_right) or 0.0001
                p0_right=(1-p1_right) or 0.0001
                entropy_left=-(p1_left*np.log2(p1_left)+p0_left*np.log2(p0_left))
                enrtopy_right=-(p1_right*np.log2(p1_right)+p0_right*np.log2(p0_right))            
                H_after.append((n_left*entropy_left+n_right*enrtopy_right)/(n_left+n_right))                
                ij_value.append((i,feature[j+1]))
    if H_after==[]:
        split=(-1,None)
    else:
        index=H_after.index(min(H_after))
        split=ij_value[index]
    return split

In [7]:
class DecisionTree:
    def __init__(self,max_depth=-1):
        self.max_depth=max_depth
        self.tree=Nodes()
    def train(self, train_data, train_labels,random_feature=-1):
        self.tree.GrowTree(self.max_depth,train_data, train_labels,random_feature)
    def predict(self,test_data):
        return np.array([prediction(self.tree,i) for i in test_data])
    def score(self,valid_data,valid_labels):
        predictions=np.array([prediction(self.tree,i) for i in valid_data])
        return (sum(predictions==valid_labels)/len(valid_labels))
    def path(self,data_point,feature_name,label_name):
        Tree=self.tree
        while Tree.split_rule[0]!=-1:
            if data_point[Tree.split_rule[0]]<Tree.split_rule[1]:
                print('"',feature_name[Tree.split_rule[0]],'"','<',Tree.split_rule[1])
                Tree=Tree.left
            else:
                print('"',feature_name[Tree.split_rule[0]],'"','>=',Tree.split_rule[1])
                Tree=Tree.right
        print('Therefore this email is',label_name[int(Tree.label)])

In [8]:
def prediction(Tree,data):
    while Tree.split_rule[0]!=-1:
        if data[Tree.split_rule[0]]<Tree.split_rule[1]:
            Tree=Tree.left
        else:
            Tree=Tree.right
    return Tree.label

In [9]:
class RandomForest:
    def __init__(self,n_trees=10,max_depth=10,sample_size=-1,n_feature=-1):
        self.n_trees=n_trees
        self.max_depth=max_depth
        self.sample_size=sample_size
        self.n_feature=n_feature
        self.trees=[]
    def train(self, train_data, train_labels):
        if self.sample_size==-1:
            self.sample_size=len(train_labels)
        if self.n_feature==-1:
            self.n_feature=int(round(np.sqrt(train_data.shape[1])))
        for i in range(self.n_trees):
            sample_ind=np.random.randint(0,len(train_labels),self.sample_size)
            sample_data=train_data[sample_ind]
            sample_label=train_labels[sample_ind]
            self.trees.append(DecisionTree(self.max_depth))
            self.trees[i].train(train_data=sample_data, train_labels=sample_label,random_feature=self.n_feature)
    def predict(self,test_data):
        pred=[]
        for i in range(self.n_trees):
            pred.append(self.trees[i].predict(test_data))
        return (np.array(pred)).mean(0).round()
    def score(self,valid_data,valid_labels):
        predictions=self.predict(valid_data)
        return (sum(predictions==valid_labels)/len(valid_labels))   
    def common_root(self,feature_name):
        rules=[]
        for i in range(self.n_trees):
            rules.append(self.trees[i].tree.split_rule)
        rule_name=[]
        rule_count=[]
        for j in set(rules):
            rule_name.append(j)
            rule_count.append(rules.count(j))
        rule_ind=np.array(rule_count).argsort()
        for j in rule_ind[::-1]:
            print('"',feature_name[rule_name[j][0]],'"','<',rule_name[j][1],':',rule_count[j],'trees')
    def common_root_split(self,data_point,feature_name):
        rules=[]
        for i in range(self.n_trees):
            rules.append(self.trees[i].tree.split_rule)
        rule_name=[]
        rule_count=[]
        for j in set(rules):
            rule_name.append(j)
            rule_count.append(rules.count(j))
        rule_ind=np.array(rule_count).argsort()
        for j in rule_ind[::-1]:
            if data_point[rule_name[j][0]]<rule_name[j][1]:
                print('"',feature_name[rule_name[j][0]],'"','<',rule_name[j][1],':',rule_count[j],'trees')
            else:
                print('"',feature_name[rule_name[j][0]],'"','>=',rule_name[j][1],':',rule_count[j],'trees')

In [10]:
Classifier=DecisionTree(max_depth=25)
Classifier.train(train_set, train_label)

In [12]:
Classifier.score(train_set, train_label)

0.82559856555215694

In [13]:
Classifier.score(valid_set,valid_label)

0.79725738396624468

In [96]:
Classifier1=RandomForest(n_trees=10,max_depth=25)
Classifier1.train(train_set, train_label)

In [97]:
Classifier1.score(train_set, train_label)

0.96408606687058329

In [98]:
Classifier1.score(valid_set,valid_label)

0.94472573839662444

In [ ]:
label_name=['ham','spam']

In [42]:
# Tree visualization.
Classifier.path(valid_set[1],feature_name,label_name)

" enron " < 1
" 2001 " < 1
" http " >= 1
" vince " < 1
Therefore this email is spam


In [43]:
Classifier.path(valid_set[2],feature_name,label_name)

" enron " < 1
" 2001 " < 1
" http " < 1
" thanks " >= 1
" this " < 5
" no " < 1
" your " >= 2
" vince " < 1
" you " < 3
" but " < 2
Therefore this email is ham


In [85]:
# Random forest visualization.
Classifier1.common_root(feature_name)

" 2000 " < 1 : 2 trees
" enron " < 1 : 2 trees
" hou " < 1 : 1 trees
" please " < 1 : 1 trees
" if " < 1 : 1 trees
" subject " < 2 : 1 trees
" 2001 " < 1 : 1 trees
" that " < 1 : 1 trees


In [86]:
Classifier1.common_root_split(valid_set[1],feature_name)

" 2000 " < 1 : 2 trees
" enron " < 1 : 2 trees
" hou " < 1 : 1 trees
" please " < 1 : 1 trees
" if " < 1 : 1 trees
" subject " < 2 : 1 trees
" 2001 " < 1 : 1 trees
" that " >= 1 : 1 trees


In [91]:
Classifier1.common_root_split(valid_set[6],feature_name)

" 2000 " < 1 : 2 trees
" enron " >= 1 : 2 trees
" hou " < 1 : 1 trees
" please " >= 1 : 1 trees
" if " < 1 : 1 trees
" subject " < 2 : 1 trees
" 2001 " < 1 : 1 trees
" that " >= 1 : 1 trees
